In [1]:
pip install pandas numpy scikit-learn matplotlib seaborn


In [3]:
import pandas as pd

df = pd.read_csv("ground_water_quality_in_west_bengal.csv", encoding="latin1")

# remove extra spaces
df.columns = df.columns.str.strip()

print(df.columns.tolist())


['STATION CODE', 'LOCATIONS', 'STATE', 'TEMPERATURE ºC : Min', 'TEMPERATURE ºC : Max', 'TEMPERATURE ºC : Mean', 'pH : Min : 6.5-8.5', 'pH : Max : 6.5-8.5', 'pH : Mean : 6.5-8.5', 'CONDUCTIVITY (µmhos/cm) : Min', 'CONDUCTIVITY (µmhos/cm) : Max', 'CONDUCTIVITY (µmhos/cm) : Mean', 'B.O.D. (mg/l) : Min : < 3 mg/l', 'B.O.D. (mg/l) : Max : < 3 mg/l', 'B.O.D. (mg/l) : Mean : < 3 mg/l', 'NITRATE- N+ NITRITE-N (mg/l) : Min', 'NITRATE- N+ NITRITE-N (mg/l) : Max', 'NITRATE- N+ NITRITE-N (mg/l) : Mean', 'FECAL COLIFORM (MPN/100ml) : Min : < 2500 MPN/100ml', 'FECAL COLIFORM (MPN/100ml) : Max : < 2500 MPN/100ml', 'FECAL COLIFORM (MPN/100ml) : Mean : < 2500 MPN/100ml', 'TOTAL COLIFORM (MPN/100ml) : Min : < 5000 MPN/100ml', 'TOTAL COLIFORM (MPN/100ml) : Max : < 5000 MPN/100ml', 'TOTAL COLIFORM (MPN/100ml) : Mean : < 5000 MPN/100ml']


In [4]:
df = df.rename(columns={
    "TEMPERATURE ºC : Mean": "TEMP_MEAN",
    "pH : Mean : 6.5-8.5": "PH_MEAN",
    "CONDUCTIVITY (µmhos/cm) : Mean": "COND_MEAN",
    "B.O.D. (mg/l) : Mean : < 3 mg/l": "BOD_MEAN",
    "D.O. (mg/l) : Mean": "DO_MEAN",
    "NITRATE- N+ NITRITE-N (mg/l) : Mean": "NITRATE_MEAN",
    "TOTAL COLIFORM (MPN/100ml) : Mean : < 5000 MPN/100ml": "TOTAL_COLIFORM_MEAN"
})


In [5]:
df["Drinkable"] = (
    (df["PH_MEAN"].between(6.5, 8.5)) &
    (df["COND_MEAN"] <= 1500) &
    (df["NITRATE_MEAN"] <= 45) &
    (df["BOD_MEAN"] <= 3) &
    (df["TOTAL_COLIFORM_MEAN"] == 0)
).astype(int)


In [6]:
def conductance_quality(val):
    if val < 750:
        return "Excellent"
    elif val <= 1500:
        return "Good"
    else:
        return "Poor"

df["Conductance_Quality"] = df["COND_MEAN"].apply(conductance_quality)


In [8]:
features = [
    "TEMP_MEAN",
    "PH_MEAN",
    "COND_MEAN",
    "BOD_MEAN",
    "NITRATE_MEAN"
]

X = df[features].fillna(df[features].median())
y = df["Drinkable"]


In [9]:
print("Features used:", features)
print("Missing columns:", set(features) - set(df.columns))


Features used: ['TEMP_MEAN', 'PH_MEAN', 'COND_MEAN', 'BOD_MEAN', 'NITRATE_MEAN']
Missing columns: set()


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load data
df = pd.read_csv("ground_water_quality_in_west_bengal.csv", encoding="latin1")
df.columns = df.columns.str.strip()

# Rename columns
df = df.rename(columns={
    "TEMPERATURE ºC : Mean": "TEMP_MEAN",
    "pH : Mean : 6.5-8.5": "PH_MEAN",
    "CONDUCTIVITY (µmhos/cm) : Mean": "COND_MEAN",
    "B.O.D. (mg/l) : Mean : < 3 mg/l": "BOD_MEAN",
    "NITRATE- N+ NITRITE-N (mg/l) : Mean": "NITRATE_MEAN",
    "TOTAL COLIFORM (MPN/100ml) : Mean : < 5000 MPN/100ml": "TOTAL_COLIFORM_MEAN"
})

# Create target
df["Drinkable"] = (
    (df["PH_MEAN"].between(6.5, 8.5)) &
    (df["COND_MEAN"] <= 1500) &
    (df["NITRATE_MEAN"] <= 45) &
    (df["BOD_MEAN"] <= 3) &
    (df["TOTAL_COLIFORM_MEAN"] == 0)
).astype(int)

# Features
features = ["TEMP_MEAN", "PH_MEAN", "COND_MEAN", "BOD_MEAN", "NITRATE_MEAN"]
X = df[features].fillna(df[features].median())
y = df["Drinkable"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Save model & scaler
joblib.dump(model, "drinkable_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("✅ Model and scaler saved successfully")


✅ Model and scaler saved successfully


In [15]:
print("\nEnter Water Quality Parameters:\n")

temp = float(input("Temperature (°C): "))
ph = float(input("pH value: "))
cond = float(input("Conductivity (µmhos/cm): "))
bod = float(input("BOD (mg/l): "))
nitrate = float(input("Nitrate (mg/l): "))
coliform = int(input("Total Coliform (MPN/100ml): "))

# -----------------------------
# DRINKABILITY CHECK
# -----------------------------
reasons = []

if not (6.5 <= ph <= 8.5):
    reasons.append("pH out of safe range")

if cond > 1500:
    reasons.append("High conductivity")

if bod > 3:
    reasons.append("High BOD")

if nitrate > 45:
    reasons.append("High nitrate")

if coliform > 0:
    reasons.append("Coliform bacteria present")

# -----------------------------
# CONDUCTANCE QUALITY
# -----------------------------
if cond < 750:
    cond_quality = "Excellent"
elif cond <= 1500:
    cond_quality = "Good"
else:
    cond_quality = "Poor"

# -----------------------------
# OUTPUT
# -----------------------------
print("\n🔍 WATER QUALITY REPORT")

print(f"Conductance Quality: {cond_quality}")

if reasons:
    print("\n❌ Water is NOT DRINKABLE")
    print("Reasons:")
    for r in reasons:
        print("-", r)
else:
    print("\n✅ Water is DRINKABLE")



Enter Water Quality Parameters:

Temperature (°C): 29
pH value: 7.1
Conductivity (µmhos/cm): 820
BOD (mg/l): 2
Nitrate (mg/l): 12
Total Coliform (MPN/100ml): 0

🔍 WATER QUALITY REPORT
Conductance Quality: Good

✅ Water is DRINKABLE


In [16]:
from google.colab import files

files.download("drinkable_model.pkl")
files.download("scaler.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>